<a href="https://colab.research.google.com/github/peterlafferty/tfp-causalimpact/blob/main/docs/colab_with_google_sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  [Causal Impact] Python Colab with Google Sheets

Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Google Sheet Template - Make A Copy

**Copy this trix to your drive, keeping the same name, or use your own trix using this as an example**
 https://docs.google.com/spreadsheets/d/1VgA7uMxYF6tsnBwgJqpj4oaLAqYPQlb8pBI0VIDGel4/


## Configure The Analysis

In [ ]:
# Google Sheet for data streams
google_sheets_url = 'https://docs.google.com/spreadsheets/d/1VgA7uMxYF6tsnBwgJqpj4oaLAqYPQlb8pBI0VIDGel4/'#@param

#date of event
event_date = '2019-10-07'#@param {type:"date"}

#How many days before intervention will be analyzed
days_before_event = 90#@param {type:"number"}

#how many days post intervention wou want to analyze (including the event date)
days_after_event = 7#@param {type:"number"}

#Number of Series used as Counterfactual (you need at least one, each in a column of the trix)
num_series = 5#@param {type:"number"}

!pip install tfp-causalimpact
import pandas as pd
import numpy as np
import causalimpact

## Load Data From Google Sheets

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open_by_url(google_sheets_url).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

df=pd.DataFrame.from_records(rows)

#get column names from first row
df.columns = df.iloc[0]

#drop row with column names
df = df.drop(df.index[0])

#reset indexes
df = df.reset_index(drop=True)

#convert numeric data to numeric types
df = df.apply(pd.to_numeric, errors='ignore')

#convert dates to dates type
df.date = pd.to_datetime(df.date, format='%Y-%m-%d', errors='ignore')

event_index = df.loc[df.date==event_date].index[0]-1

## Run The Analysis

In [ ]:
pre_period = [int(max(0,event_index-days_before_event)), int(event_index)]
post_period = [int(event_index+1), int(event_index+days_after_event)]

impact = causalimpact.fit_causalimpact(
    data=df[df.columns[1:num_series+2]],
    pre_period=pre_period,
    post_period=post_period)
print(causalimpact.summary(impact, output_format='summary'))
causalimpact.plot(impact)
print(causalimpact.summary(impact, output_format='report'))



Posterior Inference {CausalImpact}
                          Average            Cumulative
Actual                    232.0              1624.0
Prediction (s.d.)         226.3 (8.59)       1583.8 (60.13)
95% CI                    [209.8, 242.6]     [1468.4, 1698.0]

Absolute effect (s.d.)    5.7 (8.59)         40.2 (60.13)
95% CI                    [-10.6, 22.2]      [-74.0, 155.6]

Relative effect (s.d.)    2.5% (3.9%)        2.5% (4.0%)
95% CI                    [-4.4%, 10.6%]     [-4.4%, 10.6%]

Posterior tail-area probability p: 0.272
Posterior prob. of a causal effect: 72.81%

For more details run the command: summary(impact, output_format="report")

Analysis report {CausalImpact}


During the post-intervention period, the response variable had
an average value of approx. 232.0. In the absence of an
intervention, we would have expected an average response of 226.3.
The 95% interval of this counterfactual prediction is [209.8, 242.6].
Subtracting this prediction from the observed r

alt.FacetChart(...)